In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import csv
naver_Client_id = "E4zDnlSmVAnJ7dvIOUUx"
naver_Client_secret = "lLxEw7DWbB"


def get_blog_count(query,display) :
    encode_query=urllib.parse.quote(query)
    search_url="https://openapi.naver.com/v1/search/blog?query="+ encode_query +"&display=" + str(display)
    # 위의 https://~ 는 따온 API호출예제에 있음.
    request=urllib.request.Request(search_url)
    # 이렇게 하면 내가 만들었던 url 형태로 요청한다.
    # 그러나 이렇게 하면 API로부터 거부당한다.
    # 왜냐하면 ID, Secret을 주지 않았기 때문에 아래와 같이 이것을 입력해준다.
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    request.add_header("User-Agent",'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36')    

    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    
    
    # code를 받아와서 웹에서 정상적으로 접속했을 때 200번을 준다.
    if response_code is 200 :
        response_body = response.read()  # 실제로 읽어 들인다.
        response_body_dict = json.loads(response_body.decode('utf-8'))
        # 네이버에서 넘겨줄 때 json으로 넘겨주는데 이것을 전부 '딕셔너리'로 저장한다.
        
        #print('Last build date :' + str(response_body_dict['lastBuildDate']))
        # json으로 넘겨받을 것들 중에서 최종날짜를 출력
        #print('Total :' + str(response_body_dict['total']))  # 관측개수
        #print('Start :' + str(response_body_dict['start']))
        #print('Display :' + str(response_body_dict['display']))
        #이렇게 넘어온 것을 다 출력해본다.
        
        # 검색된 결과가 0 일 경우        
        if response_body_dict['total']== 0:
            blog_count = 0
        else :
            blog_total=math.ceil(response_body_dict['total']/int(display))
            # 위에서 한 번에 10개씩 출력한다고 했으니 나눠주어서 개수를 맞춰준다.
        
            if blog_total >= 1000 :
                blog_count = 1000
                # 최대 1000개로 제한이 있으니 1000개를 넘었을 때는 1000개로 지정한다.
            else : 
                blog_count = blog_total
        
            print('Blog total :' + str(blog_total))
            print('Blog count : ' + str(blog_count))
        
        one_query_information_list.append(query)
        one_query_information_list.append(str(response_body_dict['lastBuildDate']))
        one_query_information_list.append(str(response_body_dict['total']))
        one_query_information_list.append(str(blog_total))
        one_query_information_list.append(str(blog_count))
        return blog_count

In [3]:
def get_blog_post(query,display,start_index,sort):
    global no, fs
    #get_blog_count(query,display)를 정의할 때 사용했던 것과 동일하게 사용(아래 7문장)
    encode_query = urllib.parse.quote(query)
    # 동일하게 사용하나 search_url에는 옵션을 조금 더 추가해 준다.
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query +"&display=" + str(display) + "&start=" + str(start_index) + "sort=" + sort
    request=urllib.request.Request(search_url)
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    request.add_header("User-Agent",'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36')
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    #print(response_code) 이분은 윗부분까지 해서 접속이 잘되었는지를 확인하는 것.
        
    if response_code is 200:
        response_body=response.read()
        response_body_dict=json.loads(response_body.decode('utf-8'))

        for item_index in range(0,len(response_body_dict['items'])):

            try :
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['title'])
                #또한 주소가 따라오는데 이건 필요 없는 것이다.그러므로 없애준다.
                link = response_body_dict['items'][item_index]['link'].replace("amp;","")
                # description에도 마찬가지로 태그가 따라올 수 있기 때문에 제거해준다.
                description = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                #  포스트(개시글)의 주소가 아니라 블로그 자체의 링크를 가져옴.
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                #아래처럼 변환해서 가져와도 되고 그냥 주는대로 가져와도 된다.
                #post_date = datetime.datetime.strptime(response_body_dict[['item'][item_index]['postdate'],"%Y%m%d").strtime("%Y.%m.%d")

                no +=1
    
                #print("-------------------------------------------------") #각각의 게시글을 구분시킴
                #print("#"+ str(no))  #몇번째 게시글인지 출력
                #print("Title :"+ title)
                #print("Link :"+ link)
                #print("Description :"+ description)
                #print("Blogger Name :"+ blogger_name)
                #print("Blogger Link :"+ blogger_link)
                #print("Post Date :"+ post_date)
                
                if int(post_date) >= 20170101 :
                
                    post_code = requests.get(link)
                    post_text = post_code.text
                    post_soup = BeautifulSoup(post_text,'lxml')
    
                    for mainFrame in post_soup.select('iframe#mainFrame'):
                        blog_post_url = "http://blog.naver.com"+mainFrame.get('src')
                        blog_post_code = requests.get(blog_post_url)
                        blog_post_text = blog_post_code.text
                        blog_post_soup = BeautifulSoup(blog_post_text,'lxml')
                        
                        for selector in ['div#postViewArea','.se-main-container','div#contentDiv','div#se_component_wrap sect_dsc __se_component_area']:
                            for pc in blog_post_soup.select(selector):
                                if len(str(pc.get_text())) > 2:
                                    pc_text = pc.get_text()
                                    pc_text2 = str(pc_text).replace("\n",'')  #컨텐트를 string으로 캐스팅해서 가져온다.
                                    pc_text2 = str(pc_text).replace("\u200b",'')
                                    #print(pc_text2)
                                
                                else:
                                    continue
                                #print([title,post_date,blogger_name,description,link,pc_text2])
                                csv_writer.writerow([title,post_date,blogger_name,description,link,pc_text2])
                                
                         
            #만약에 못할 경우 그냥 넘어가는 코드.
            except: 
                item_index+=1
                print("■■■■■■저장할 수 없음.■■■■■■")

In [4]:
#query_pd=pd.read_csv("네이버Blog 키워드.csv",encoding='cp949')
#query_list=query_pd['품명'].values.tolist()
query_list=['창원']

In [5]:
   

no = 0           # 몇 개의 포스트를 저장했는지 세기 위한 index
#query = "대구" # 검색을 원하는 문자열로써 UTF-8로 인코딩한다.
display = 10    # 검색 결과 출력 건수 지정, 10(기본값), 100(최대값)
start = 1        # 검색 시작 위치로 최대 1000까지 가능
sort = 'date'    # 정렬 옵션 : sim(유사도순, 기본값), date(날짜순)

all_query_information_pd=pd.DataFrame(columns=['query','Last build date','Total','Blog total','Blog count','stored data'])     
for query in query_list :
    print(query+"시작")
    fs = open("./NaverBlogData_v0.2/v0.2_"+query + ".csv",'w',encoding='utf-8',newline="")
    csv_writer=csv.writer(fs)
    csv_writer.writerow(["title","post_date","blogger_name","description","link","content"])
    one_query_information_list=[]
    
    blog_count = get_blog_count(query,display)
    
    for start_index in range(start,blog_count + 1, display):
        get_blog_post(query,display,start_index,sort)  
    
    fs.close()
    data=pd.read_csv("./NaverBlogData_v0.2/v0.2_"+query+".csv",encoding='utf-8')
    one_query_information_list.append(len(data))
    all_query_information_pd.loc[len(all_query_information_pd)]=one_query_information_list
    print(all_query_information_pd)
    
all_query_information_pd.to_csv("./NaverBlogData_v0.2/"+"v0.2_all_query_information.csv",encoding='utf-8',index=False)

창원시작
Blog total :361663
Blog count : 1000
  query                  Last build date    Total Blog total Blog count  \
0    창원  Thu, 29 Aug 2019 15:29:42 +0900  3616626     361663       1000   

  stored data  
0         962  
